В этом блокоте приведено использование обученной модели на детекции труб на предоставленном нас наборе данных

In [1]:
import cv2
import matplotlib.pyplot as plt
import os
import shutil
import numpy as np
from PIL import Image, ImageFilter

Тут подгружаем YOLO и проверяем, задейтвуется ли видеокарта и CUDA

In [2]:
from ultralytics import YOLO
from IPython.display import display, Image
from IPython import display
display.clear_output()
!yolo checks



Ultralytics YOLOv8.2.18 рџљЂ Python-3.11.4 torch-2.0.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3070 Laptop GPU, 8192MiB)
Setup complete вњ… (8 CPUs, 15.7 GB RAM, 372.1/471.6 GB disk)

OS                  Windows-10-10.0.19043-SP0
Environment         Windows
Python              3.11.4
Install             pip
RAM                 15.74 GB
CPU                 11th Gen Intel Core(TM) i7-11370H 3.30GHz
CUDA                11.8

matplotlib          вњ… 3.7.2>=3.3.0
opencv-python       вњ… 4.9.0.80>=4.6.0
pillow              вњ… 9.5.0>=7.1.2
pyyaml              вњ… 6.0.1>=5.3.1
requests            вњ… 2.31.0>=2.23.0
scipy               вњ… 1.11.1>=1.4.1
torch               вњ… 2.0.1+cu118>=1.8.0
torchvision         вњ… 0.15.2+cu118>=0.9.0
tqdm                вњ… 4.66.1>=4.64.0
psutil              вњ… 5.9.5
py-cpuinfo          вњ… 9.0.0
thop                вњ… 0.1.1-2209072238>=0.1.1
pandas              вњ… 2.0.3>=1.1.4
seaborn             вњ… 0.12.2>=0.11.0


в model_hope указывается путь по весов модели
В path путь в файлам, которые мы хотим "кормить" можели

In [3]:
model_hope = YOLO(r'C:/Users/PyatnitskayaOT/PycharmProjects/yolov8/runs/detect/train3/weights/best.pt')

In [ ]:
path = r'C:\Users\PyatnitskayaOT\PycharmProjects\yolov8\hakaton_05_24\cap'

"Кормим" модель картинками из папки нужным нам образом

In [20]:
result = model_hope.predict(path, save=True, conf=0.8, show_conf=False, show_labels=False)


image 1/20 C:\Users\PyatnitskayaOT\PycharmProjects\yolov8\hakaton_05_24\cap\WhatsApp Image 2024-05-19 at 16.27.18.jpeg: 480x640 105 0s, 21.0ms
image 2/20 C:\Users\PyatnitskayaOT\PycharmProjects\yolov8\hakaton_05_24\cap\WhatsApp Image 2024-05-19 at 16.27.19 (1).jpeg: 480x640 45 0s, 18.9ms
image 3/20 C:\Users\PyatnitskayaOT\PycharmProjects\yolov8\hakaton_05_24\cap\WhatsApp Image 2024-05-19 at 16.27.19.jpeg: 480x640 124 0s, 18.0ms
image 4/20 C:\Users\PyatnitskayaOT\PycharmProjects\yolov8\hakaton_05_24\cap\WhatsApp Image 2024-05-19 at 16.27.20 (1).jpeg: 480x640 45 0s, 18.0ms
image 5/20 C:\Users\PyatnitskayaOT\PycharmProjects\yolov8\hakaton_05_24\cap\WhatsApp Image 2024-05-19 at 16.27.20.jpeg: 480x640 131 0s, 18.0ms
image 6/20 C:\Users\PyatnitskayaOT\PycharmProjects\yolov8\hakaton_05_24\cap\WhatsApp Image 2024-05-19 at 16.27.21 (1).jpeg: 480x640 108 0s, 18.0ms
image 7/20 C:\Users\PyatnitskayaOT\PycharmProjects\yolov8\hakaton_05_24\cap\WhatsApp Image 2024-05-19 at 16.27.21.jpeg: 640x480 81 

In [124]:
#вывод количества обнаруженных дыр на одной конретной фотке
result[10].boxes.shape[0]

159

In [23]:
#цикл по result, путь к картинке + количество обнаруженных на ней дырок
for n in range(0,20):
    print(result[n].path)
    print(result[n].boxes.shape[0])

C:\Users\PyatnitskayaOT\PycharmProjects\yolov8\hakaton_05_24\cap\WhatsApp Image 2024-05-19 at 16.27.18.jpeg
105
C:\Users\PyatnitskayaOT\PycharmProjects\yolov8\hakaton_05_24\cap\WhatsApp Image 2024-05-19 at 16.27.19 (1).jpeg
45
C:\Users\PyatnitskayaOT\PycharmProjects\yolov8\hakaton_05_24\cap\WhatsApp Image 2024-05-19 at 16.27.19.jpeg
124
C:\Users\PyatnitskayaOT\PycharmProjects\yolov8\hakaton_05_24\cap\WhatsApp Image 2024-05-19 at 16.27.20 (1).jpeg
45
C:\Users\PyatnitskayaOT\PycharmProjects\yolov8\hakaton_05_24\cap\WhatsApp Image 2024-05-19 at 16.27.20.jpeg
131
C:\Users\PyatnitskayaOT\PycharmProjects\yolov8\hakaton_05_24\cap\WhatsApp Image 2024-05-19 at 16.27.21 (1).jpeg
108
C:\Users\PyatnitskayaOT\PycharmProjects\yolov8\hakaton_05_24\cap\WhatsApp Image 2024-05-19 at 16.27.21.jpeg
81
C:\Users\PyatnitskayaOT\PycharmProjects\yolov8\hakaton_05_24\cap\WhatsApp Image 2024-05-19 at 16.27.22.jpeg
254
C:\Users\PyatnitskayaOT\PycharmProjects\yolov8\hakaton_05_24\cap\WhatsApp Image 2024-05-19 at 1

Тут мы "дорисовываем" на картинках с Bounding boxes их количество

In [24]:
# path_predict - папка с картинками
# path_writr - куда сохраняем снимки с дорисованнным количеством 
path_predict = r'C:\Users\PyatnitskayaOT\PycharmProjects\yolov8\hakaton_05_24\runs\detect\predict'
path_write = r'C:\Users\PyatnitskayaOT\PycharmProjects\yolov8\hakaton_05_24\runs\detect\new'
# Перебираем все файлы в папке predict и рисуем на них количество дырок, сохраняем в new
n = 0
for file in os.listdir(path_predict):
    # Получаем полный путь к файлу
    full_path = os.path.join(path_predict, file)
    #выводим имя файла, значение счетсика, соответсвующее количество из result
    print(file)
    print(n)
    print(result[n].path)
    #загружаем картинку
    frame = cv2.imread(full_path)
    #рисуем на картинке в левом верхнем углу белую линию
    frame1 = cv2.line(frame,(0,52),(200,52),(255,255,255),110) 
    #на нарисованной белой линии пишем количество объектов на картинке из result
    frame1 = cv2.putText(frame1, str(result[n].boxes.shape[0]), (0,100), cv2.FONT_HERSHEY_SIMPLEX, 4, (0,0, 255), 5)
    #сохраняем новую картинку
    cv2.imwrite(os.path.join(path_write, str(n)+ '.jpg'), frame1)
    #увеличиваем значение счетчика
    n+= 1


WhatsApp Image 2024-05-19 at 16.27.18.jpeg
0
C:\Users\PyatnitskayaOT\PycharmProjects\yolov8\hakaton_05_24\cap\WhatsApp Image 2024-05-19 at 16.27.18.jpeg
WhatsApp Image 2024-05-19 at 16.27.19 (1).jpeg
1
C:\Users\PyatnitskayaOT\PycharmProjects\yolov8\hakaton_05_24\cap\WhatsApp Image 2024-05-19 at 16.27.19 (1).jpeg
WhatsApp Image 2024-05-19 at 16.27.19.jpeg
2
C:\Users\PyatnitskayaOT\PycharmProjects\yolov8\hakaton_05_24\cap\WhatsApp Image 2024-05-19 at 16.27.19.jpeg
WhatsApp Image 2024-05-19 at 16.27.20 (1).jpeg
3
C:\Users\PyatnitskayaOT\PycharmProjects\yolov8\hakaton_05_24\cap\WhatsApp Image 2024-05-19 at 16.27.20 (1).jpeg
WhatsApp Image 2024-05-19 at 16.27.20.jpeg
4
C:\Users\PyatnitskayaOT\PycharmProjects\yolov8\hakaton_05_24\cap\WhatsApp Image 2024-05-19 at 16.27.20.jpeg
WhatsApp Image 2024-05-19 at 16.27.21 (1).jpeg
5
C:\Users\PyatnitskayaOT\PycharmProjects\yolov8\hakaton_05_24\cap\WhatsApp Image 2024-05-19 at 16.27.21 (1).jpeg
WhatsApp Image 2024-05-19 at 16.27.21.jpeg
6
C:\Users\Pyat

Далее у нас код с подсчетом труб в конкретном участве кадра на видео
Имя нужного видео файла указываетя в  cap
В region_points задем область интеереса
В video_writer указваем име выходного файла

In [6]:
import cv2
from ultralytics import YOLO, solutions

model = model_hope
cap = cv2.VideoCapture("video0.mkv")
assert cap.isOpened(), "Error reading video file"
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))
print(w,h)
# Define region points as a polygon
#(x1,y1)(x1,y2)(x2,y2)(x2,y1)
region_points = [(500, 470), (500, 678), (950, 678), (950, 470)]

# Video writer
video_writer = cv2.VideoWriter("output0.mp4", cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

# Init Object Counter
counter = solutions.ObjectCounter(
    view_img=True,
    reg_pts=region_points,
    classes_names=model.names,
    draw_tracks=True,
    line_thickness=2,
)
#cap.isOpened()
while True:
    success, im0 = cap.read()
    if not success:
        print("Video frame is empty or video processing has been successfully completed.")
        break
    tracks = model.track(im0, persist=True, show=False)

    im0 = counter.start_counting(im0, tracks)
    video_writer.write(im0)

cap.release()
video_writer.release()
cv2.destroyAllWindows()

1280 720
Polygon Counter Initiated.

0: 384x640 3 0s, 21.0ms
Speed: 5.0ms preprocess, 21.0ms inference, 5.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 0s, 16.0ms
Speed: 5.0ms preprocess, 16.0ms inference, 7.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 0s, 17.0ms
Speed: 6.0ms preprocess, 17.0ms inference, 7.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 0s, 17.0ms
Speed: 5.0ms preprocess, 17.0ms inference, 5.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 0s, 17.0ms
Speed: 4.0ms preprocess, 17.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 0s, 16.0ms
Speed: 3.0ms preprocess, 16.0ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 0s, 16.0ms
Speed: 6.0ms preprocess, 16.0ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 0s, 20.0ms
Speed: 5.0ms preprocess, 20.0ms inference, 3.0ms postprocess per image at shape (1, 3

KeyboardInterrupt: 

In [3]:
#линия
import cv2
from ultralytics import YOLO, solutions

model = YOLO(r'C:/Users/PyatnitskayaOT/PycharmProjects/yolov8/runs/detect/train3/weights/best.pt')
cap = cv2.VideoCapture("video2.mp4")
assert cap.isOpened(), "Error reading video file"
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))
print(w,h)
#(x1,y1)(x1,y2)(x2,y2)(x2,y1)s
#line_points = [(3, 255),(3, 750), (800, 900),(1100,900),(1100,255)]
# Define line points
line_points = [(675, 525),(675, 1000),(1400, 1000),(1400,525)]

# Video writer
video_writer = cv2.VideoWriter("output2.mp4", cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

# Init Object Counter
counter = solutions.ObjectCounter(
    view_img=True,
    reg_pts=line_points,
    classes_names=model.names,
    draw_tracks=True,
    line_thickness=3,
    region_thickness = 10
    
)

while cap.isOpened():
    success, im0 = cap.read()
    if not success:
        print("Video frame is empty or video processing has been successfully completed.")
        break
    tracks = model.track(im0, persist=True, show=False, conf=0.2, verbose=False)

    im0 = counter.start_counting(im0, tracks)
    video_writer.write(im0)

cap.release()
video_writer.release()
cv2.destroyAllWindows()

1920 1080
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
